In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk import pos_tag, map_tag
from nltk.corpus import sentiwordnet as swn
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from textblob import TextBlob
from afinn import Afinn

In [2]:
df=pd.read_csv('health_news.csv')

In [3]:
df.head()

,Unnamed: 0,numbers,time,tweets
0,0,585978391360221184,Thu Apr 09 01:31:50 +0000 2015,Breast cancer risk test devised http://bbc.in/...
1,1,585947808772960257,Wed Apr 08 23:30:18 +0000 2015,GP workload harming care - BMA poll http://bbc...
2,2,585947807816650752,Wed Apr 08 23:30:18 +0000 2015,Short people's 'heart risk greater' http://bbc...
3,3,585866060991078401,Wed Apr 08 18:05:28 +0000 2015,New approach against HIV 'promising' http://bb...
4,4,585794106170839041,Wed Apr 08 13:19:33 +0000 2015,Coalition 'undermined NHS' - doctors http://bb...


In [4]:
df.drop(['numbers', 'time'], axis=1, inplace=True)

In [5]:
df.shape

(55230, 2)

In [6]:
df.isnull().values.any()

False

In [7]:
# import re
# import string
from Scripts.preprocessor import TwitterPreprocessor

In [8]:
df['clean_text'] = df.tweets.apply(lambda x: TwitterPreprocessor(x).ml_preprocess().text)

In [9]:
df.head()

,Unnamed: 0,tweets,clean_text
0,0,Breast cancer risk test devised http://bbc.in/...,breast cancer risk test devised
1,1,GP workload harming care - BMA poll http://bbc...,gp workload harming care bma poll
2,2,Short people's 'heart risk greater' http://bbc...,short people heart risk greater
3,3,New approach against HIV 'promising' http://bb...,new approach hiv promising
4,4,Coalition 'undermined NHS' - doctors http://bb...,coalition undermined nhs doctors


In [10]:
import boto3

In [21]:
comprehend = boto3.client(service_name='comprehend', region_name="region")
text = "There is smoke in San Francisco and it makes me angry"
comprehend.detect_sentiment(Text=text, LanguageCode='en')

EndpointConnectionError: Could not connect to the endpoint URL: "https://comprehend.region.amazonaws.com/"

In [11]:
def create_sentiment_aws(row):
    """Uses AWS Comprehend to Create Sentiments on a DataFrame"""

    try:
      comprehend = boto3.client(service_name='comprehend', region_name="us-east-2")
      payload = comprehend.detect_sentiment(Text=row, LanguageCode='en')  
      sentiment = payload['Sentiment']
    except Exception:
      print("Size exceeded:  Fail")
      return None
    return sentiment

def apply_sentiment_aws(df, column="clean_text"):
    """Uses Pandas Apply to Create Sentiment Analysis"""

    df['Sentiment'] = df[column].apply(create_sentiment_aws)
    return df

In [15]:
df1 = df[:50]
df1.shape

(50, 3)

In [16]:
df1_ = apply_sentiment_aws(df1)

Size exceeded:  Fail
Size exceeded:  Fail
Size exceeded:  Fail
Size exceeded:  Fail
Size exceeded:  Fail
Size exceeded:  Fail
Size exceeded:  Fail


KeyboardInterrupt: 